In [43]:
import enum
from typing import Optional, List

from kor import from_pydantic
from pydantic import BaseModel, Field
from kor.extraction import create_extraction_chain
from langchain.chat_models import ChatOpenAI

In [44]:

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
)

In [45]:
class Action(enum.Enum):
    play = "play"
    stop = "stop"
    previous = "previous"
    next_ = "next"


class MusicRequest(BaseModel):
    song: Optional[List[str]] = Field(
        description="The song(s) that the user would like to be played."
    )
    album: Optional[List[str]] = Field(
        description="The album(s) that the user would like to be played."
    )
    artist: Optional[List[str]] = Field(
        description="The artist(s) whose music the user would like to hear.",
        examples=[("Songs by paul simon", "paul simon")],
    )
    action: Optional[Action] = Field(
        description="The action that should be taken; one of `play`, `stop`, `next`, `previous`",
        examples=[
            ("Please stop the music", "stop"),
            ("play something", "play"),
            ("play a song", "play"),
            ("next song", "next"),
        ],
    )
    volume: Optional[float] = Field(
        description="Set the volume",
    )

In [46]:
schema, validator = from_pydantic(MusicRequest, description="Music recorder")
schema

Object(id='musicrequest', description='Music recorder', many=False, attributes=[Text(id='song', description='The song(s) that the user would like to be played.', many=True, examples=()), Text(id='album', description='The album(s) that the user would like to be played.', many=True, examples=()), Text(id='artist', description='The artist(s) whose music the user would like to hear.', many=True, examples=[('Songs by paul simon', 'paul simon')]), Selection(id='action', description='The action that should be taken; one of `play`, `stop`, `next`, `previous`', many=False, options=[Option(id='play', description='', many=False, examples=()), Option(id='stop', description='', many=False, examples=()), Option(id='previous', description='', many=False, examples=()), Option(id='next', description='', many=False, examples=())], examples=[('Please stop the music', 'stop'), ('play something', 'play'), ('play a song', 'play'), ('next song', 'next')], null_examples=()), Number(id='volume', description='S

# TypeScript

In [47]:
from kor import TypeScriptDescriptor

In [48]:
descriptor = TypeScriptDescriptor()

In [49]:
print(descriptor.describe(schema))

```TypeScript

musicrequest: { // Music recorder
 song: Array<string> // The song(s) that the user would like to be played.
 album: Array<string> // The album(s) that the user would like to be played.
 artist: Array<string> // The artist(s) whose music the user would like to hear.
 action: "play" | "stop" | "previous" | "next" // The action that should be taken; one of `play`, `stop`, `next`, `previous`
 volume: number // Set the volume
}
```



In [50]:
chain = create_extraction_chain(llm, schema, type_descriptor=descriptor, encoder_or_encoder_class="json")

print(chain.prompt.format_prompt(text='hello').to_string())

Your goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

```TypeScript

musicrequest: { // Music recorder
 song: Array<string> // The song(s) that the user would like to be played.
 album: Array<string> // The album(s) that the user would like to be played.
 artist: Array<string> // The artist(s) whose music the user would like to hear.
 action: "play" | "stop" | "previous" | "next" // The action that should be taken; one of `play`, `stop`, `next`, `previous`
 volume: number // Set the volume
}
```


Please output the extracted information in JSON format. Do not output anything except for the extracted information. Do not add any clarifying information. Do not add any fields that are not in the schema. If the text contains attributes that do not appear in the schema, please ignor

# BulletPoint

In [51]:
from kor import BulletPointDescriptor

In [52]:
descriptor = BulletPointDescriptor()

In [53]:
print(descriptor.describe(schema))

* musicrequest: Object # Music recorder
*  song: Text # The song(s) that the user would like to be played.
*  album: Text # The album(s) that the user would like to be played.
*  artist: Text # The artist(s) whose music the user would like to hear.
*  action: Selection # The action that should be taken; one of `play`, `stop`, `next`, `previous`
*  volume: Number # Set the volume


In [54]:
chain = create_extraction_chain(llm, schema, type_descriptor=descriptor, encoder_or_encoder_class="json")

print(chain.prompt.format_prompt(text='hello').to_string())

Your goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

* musicrequest: Object # Music recorder
*  song: Text # The song(s) that the user would like to be played.
*  album: Text # The album(s) that the user would like to be played.
*  artist: Text # The artist(s) whose music the user would like to hear.
*  action: Selection # The action that should be taken; one of `play`, `stop`, `next`, `previous`
*  volume: Number # Set the volume

Please output the extracted information in JSON format. Do not output anything except for the extracted information. Do not add any clarifying information. Do not add any fields that are not in the schema. If the text contains attributes that do not appear in the schema, please ignore them. All output must be in JSON format and follow the schema sp

# Custom
Here’s an example on how to define your own type-description.

In [55]:
from typing import Any

from kor import TypeScriptDescriptor, Object
from kor.nodes import AbstractSchemaNode

In [56]:
class MeowDescriptor(TypeScriptDescriptor):
    def visit_default(self, node: "AbstractSchemaNode", **kwargs: Any) -> List[str]:
        """Default action for a node."""
        depth = kwargs["depth"]
        space = " " + depth * " ~(^._.)" + " "
        return [f"{space}{node.id}: {node.__class__.__name__} # {node.description}"]

    def visit_object(self, node: Object, **kwargs: Any) -> List[str]:
        """Visit an object node."""
        depth = kwargs["depth"]
        code_lines = self.visit_default(node, depth=depth)
        for child in node.attributes:
            code_lines.extend(child.accept(self, depth=depth + 1))
        return code_lines

    def describe(self, node: Object) -> str:
        """Describe the type of the given node."""
        code_lines = node.accept(self, depth=0)
        return "\n".join(code_lines)

In [57]:
print(MeowDescriptor().describe(schema))

  musicrequest: Object # Music recorder
  ~(^._.) song: Text # The song(s) that the user would like to be played.
  ~(^._.) album: Text # The album(s) that the user would like to be played.
  ~(^._.) artist: Text # The artist(s) whose music the user would like to hear.
  ~(^._.) action: Selection # The action that should be taken; one of `play`, `stop`, `next`, `previous`
  ~(^._.) volume: Number # Set the volume
